In [9]:
from os import listdir, getcwd
from os.path import join
from RedirectStdStreams import RedirectStdStreams
from io import StringIO
import pandas as pd

In [10]:
DATA_DIR = 'STACIA_QA'
DIR = join(getcwd(), DATA_DIR)

In [11]:
files = listdir(DATA_DIR)

In [12]:
def startswith(y):
    return lambda x: x.startswith(y)

In [13]:
cs_files = [f for f in filter(startswith('cs'), files)]
stat_files = [f for f in filter(startswith('stat'), files)]
club_files = [f for f in filter(startswith('club'), files)]

In [14]:
# encodings: https://stackoverflow.com/q/19699367/5411712
# list-of-encodings:
# https://docs.python.org/3/library/codecs.html#standard-encodings
for fname in cs_files:
    fpath = join(DIR, fname)
    warnings = StringIO()  # to catch the warnings
    with RedirectStdStreams(stderr=warnings):
        pd.read_csv(fpath,
                    sep='|',
                    encoding='latin_1',
                    header=None,
                    error_bad_lines=False,
                    warn_bad_lines=True)
    warnings.seek(0)
    warns = warnings.read()
    warnings.close()

    if len(warns) > 10:
        # for whatever reason the warnings look like "b'Skipping...'\n"
        # so chop off the weird stuff on the edges
        warns = warns[2:len(warns)-2]
        warns = warns.replace('\\n', '\n')
        with StringIO(warns) as w:
            warn_df = pd.read_csv(w, sep='\n', header=None)
            warn_df.columns = ['warns']
            # https://chrisalbon.com/python/data_wrangling/pandas_regex_to_create_columns/
            warn_df['skipped_lines'] = warn_df['warns'].str.extract(
                'Skipping line (\d+)', expand=True)
            warn_df['expected_cols'] = warn_df['warns'].str.extract(
                'expected (\d+)', expand=True)
            warn_df['actual_cols'] = warn_df['warns'].str.extract(
                'saw (\d+)', expand=True)

            with open(fpath, encoding='latin_1') as f:
                lines = f.readlines()
                line_num = int(warn_df['skipped_lines'][0])
                bad_line = lines[line_num - 1]  # because lines are 1-indexed
                print('bad_line is #', line_num, '...', bad_line)

    print(fname, "done", "....on to the next one....")

cs8.txt done ....on to the next one....
cs9.txt done ....on to the next one....
cs1.txt done ....on to the next one....
bad_line is # 14 ... A1|Are there any upcoming events for the [Stat-Club] club in the STAT department?|(No. | Yes. The next event is [Stat-Event] on [Date].)

cs2.txt done ....on to the next one....
cs3.txt done ....on to the next one....
cs7.txt done ....on to the next one....
cs6.txt done ....on to the next one....
bad_line is # 14 ... A1|Are there any upcoming events for the [Stat-Club] club in the STAT department?|(No. | Yes. The next event is [Stat-Event] on [Date].)

cs4.txt done ....on to the next one....
cs5.txt done ....on to the next one....
cs15.txt done ....on to the next one....
cs14.txt done ....on to the next one....
cs10.txt done ....on to the next one....
cs11.txt done ....on to the next one....
cs13.txt done ....on to the next one....
bad_line is # 31 ... G1|How many units of tech electives are required for a [Major| Minor]?|Number

cs12.txt done ...

In [15]:
for fname in stat_files:
    warnings = StringIO()  # to catch the warnings
    with RedirectStdStreams(stderr=warnings):
        pd.read_csv(join(DIR, fname),
                    sep='|',
                    encoding='latin_1',
                    header=None,
                    error_bad_lines=False,
                    warn_bad_lines=True)
    warnings.seek(0)
#    print("warnings??",warnings.read())
    print(fname, "done", "....on to the next one....")

stat1.txt done ....on to the next one....


In [16]:
for fname in club_files:
    warnings = StringIO()  # to catch the warnings
    with RedirectStdStreams(stderr=warnings):
        pd.read_csv(join(DIR, fname),
                    sep='|',
                    encoding='latin_1',
                    header=None,
                    error_bad_lines=False,
                    warn_bad_lines=True)
    warnings.seek(0)
#    print("warnings??",warnings.read())
    print(fname, "done", "....on to the next one....")


clubs.txt done ....on to the next one....
